In [1]:
import numpy as np
import pandas as pd 
from deepctr.models import DIN
from deepctr.feature_column import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
from sklearn.utils import shuffle

In [2]:
# 读取数据
samples_data = pd.read_csv("./data/criteo_train_lessTags.txt")
samples_data = shuffle(samples_data)
samples_data.head()

,label,I1,I2,I3,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11
430412,1,2200.0,1156,2200.0,0,1,1,0,0,0,0,0,0,1332,4866
2555534,0,900.0,12381,900.0,0,0,0,0,0,0,1,0,1,1212,2703
1145222,0,2800.0,675,2800.0,1,0,1,0,1,0,0,0,0,263,196
1891978,1,2500.0,804,2500.0,1,0,1,0,0,0,1,0,0,27,2822
1515007,1,2000.0,2646,2000.0,0,0,1,0,0,0,0,0,0,1634,5530


In [3]:
for i in range(1,10):
    print('C'+str(i),samples_data['C'+str(i)].sum())

C1 541607
C2 405609
C3 606150
C4 271846
C5 161517
C6 79194
C7 1209893
C8 741835
C9 361755


In [4]:
samples_data['user_id']=samples_data['C10']
samples_data['movie_id']=samples_data['C11']
samples_data['movie_type_id']=0
samples_data.loc[samples_data['C7']==1,'movie_type_id']=1
samples_data.loc[(samples_data['C7']==0)&(samples_data['C8']==1),'movie_type_id']=2
samples_data.loc[(samples_data['C7']==0)&(samples_data['C8']==0)&(samples_data['C6']==1),'movie_type_id']=3
samples_data.loc[(samples_data['C7']==0)&(samples_data['C8']==0)&(samples_data['C6']==0)&(samples_data['C3']==1),'movie_type_id']=4
samples_data.head()

,label,I1,I2,I3,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,user_id,movie_id,movie_type_id
430412,1,2200.0,1156,2200.0,0,1,1,0,0,0,0,0,0,1332,4866,1332,4866,4
2555534,0,900.0,12381,900.0,0,0,0,0,0,0,1,0,1,1212,2703,1212,2703,1
1145222,0,2800.0,675,2800.0,1,0,1,0,1,0,0,0,0,263,196,263,196,4
1891978,1,2500.0,804,2500.0,1,0,1,0,0,0,1,0,0,27,2822,27,2822,1
1515007,1,2000.0,2646,2000.0,0,0,1,0,0,0,0,0,0,1634,5530,1634,5530,4


In [6]:
hist_click =samples_data[['user_id', 'movie_id']].groupby('user_id').agg({list}).reset_index()
his_behavior_df = pd.DataFrame()
his_behavior_df['user_id'] = hist_click['user_id']
his_behavior_df['hist_movie_id'] = hist_click['movie_id']
#minlen=min(his_behavior_df['hist_movie_id'].map(lambda x:len(x)))
#his_behavior_df['hist_movie_id']=his_behavior_df['hist_movie_id'].map(lambda x:x[:minlen])#不能直接drop,否则报错，很奇怪，此处是截断操作
# 通过下面的padding可以做到长度都为50，就不需要截断了
his_behavior_df.head()

,user_id,hist_movie_id
0,0,"[2310, 3418, 2586, 2334, 2686, 2859, 4879, 25,..."
1,1,"[2644, 2792, 2792, 2602, 2431, 2269, 2792, 165..."
2,2,"[2586, 2586, 2586, 2586, 2586, 2586, 2586, 258..."
3,3,"[1178, 5337, 109, 2732, 2656, 2586, 2656, 2656..."
4,4,"[146, 2822, 5140, 4233, 2510, 2482, 2510, 1297..."


In [43]:
#his_behavior_df.drop(his_behavior_df[his_behavior_df['hist_movie_id'].map(lambda x:len(x)<50)].index,inplace=True)
#his_behavior_df['hist_movie_id']=his_behavior_df['hist_movie_id'].map(lambda x:x[:50])

In [62]:
min(his_behavior_df['hist_movie_id'].map(lambda x:len(x)))

6

In [12]:
samples_data=samples_data.merge(his_behavior_df,on='user_id',how='left')
samples_data.head()

,label,I1,I2,I3,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,user_id,movie_id,movie_type_id,hist_movie_id
0,1,2200.0,1156,2200.0,0,1,1,0,0,0,0,0,0,1332,4866,1332,4866,4,"[4866, 3800, 4212, 1152, 2695, 4212, 4905, 523..."
1,0,900.0,12381,900.0,0,0,0,0,0,0,1,0,1,1212,2703,1212,2703,1,"[2703, 2788, 2166, 2462, 2649, 2721, 2703, 290..."
2,0,2800.0,675,2800.0,1,0,1,0,1,0,0,0,0,263,196,263,196,4,"[196, 2812, 2812, 2812, 2586, 1555, 1690, 3881..."
3,1,2500.0,804,2500.0,1,0,1,0,0,0,1,0,0,27,2822,27,2822,1,"[2822, 2429, 2784, 1571, 2719, 2696, 2429, 278..."
4,1,2000.0,2646,2000.0,0,0,1,0,0,0,0,0,0,1634,5530,1634,5530,4,"[5530, 2771, 2724, 5345, 1819, 2724, 2771, 277..."


# 使用函数设置特征

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 数据准备函数
def get_din_feats_columns(df, dense_fea, sparse_fea, behavior_fea, hist_behavior_fea, emb_dim=32, max_len=50):
    """
    数据准备函数:
    df: 数据集
    dense_fea: 数值型特征列
    sparse_fea: 离散型特征列
    behavior_fea: 用户的候选行为特征列
    his_behavior_fea: 用户的历史行为特征列
    embedding_dim: embedding的维度， 这里为了简单， 统一把离散型特征列采用一样的隐向量维度
    max_len: 用户序列的最大长度
    """
    
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
    
    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]
    
    var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=df['movie_id'].nunique() + 1,
                            embedding_dim=emb_dim, embedding_name='movie_id'), maxlen=max_len) for feat in hist_behavior_fea]
    #把短的seq直接padding一下，不需要截断
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    
    # 建立x, x是一个字典的形式
    x = {}
    for name in get_feature_names(dnn_feature_columns):
        if name in hist_behavior_fea:
            # 这是历史行为序列
            his_list = [l for l in df[name]]
            x[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
        else:
            x[name] = df[name].values
    
    return x, dnn_feature_columns

In [25]:
# 稀疏特征
sparse_fea = ['user_id', 'movie_id', 'movie_type_id']
# 稠密特征
dense_fea = ["I1", "I2"]
# 行为特征列表，表示的是基础特征
behavior_feature_list = ['movie_id']
# 行为序列特征
behavior_seq_feature_list = ['hist_movie_id']
target=['label']
X = samples_data[["user_id", "I1", "I2", "hist_movie_id", "movie_id", "movie_type_id"]]
y = samples_data["label"]
X_train,feature_columns=get_din_feats_columns(X,dense_fea,sparse_fea,behavior_feature_list,behavior_seq_feature_list)

In [26]:
model = DIN(feature_columns, behavior_feature_list, behavior_seq_feature_list)

model.compile('adam',loss='binary_crossentropy', metrics=['AUC'])

history=model.fit(X_train, samples_data[target].values, batch_size=64, epochs=1, validation_split=0.2, )

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer_2/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer_2/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
35014/35014 [==============================] - 332s 9ms/step - loss: 0.2652 - auc: 0.9506 - val_loss: 0.0833 - val_auc: 0.9961


# 手动设置特征

In [64]:
X = samples_data[["user_id", "I1", "I2", "hist_movie_id", "movie_id", "movie_type_id"]]
y = samples_data["label"]

X_train = {"user_id": np.array(X["user_id"]), \
            "hist_movie_id": np.array([[int(i) for i in l] for l in X["hist_movie_id"]]), \
            "I1": np.array(X["I1"]), \
            "I2": np.array(X["I2"]), \
            "movie_id": np.array(X["movie_id"]), \
            "movie_type_id": np.array(X["movie_type_id"])}

y_train = np.array(y)

feature_columns = [SparseFeat('user_id', max(samples_data["user_id"])+1, embedding_dim=8), 
                        SparseFeat('movie_id', max(samples_data["movie_id"])+1, embedding_dim=8),
                        SparseFeat('movie_type_id', max(samples_data["movie_type_id"])+1, embedding_dim=8),
                        DenseFeat('I1', 1),DenseFeat('I2', 1)]

feature_columns += [VarLenSparseFeat(SparseFeat('hist_movie_id', vocabulary_size=max(samples_data["movie_id"])+1, embedding_dim=8), maxlen=min(his_behavior_df['hist_movie_id'].map(lambda x:len(x))))]# maxlen为序列最短值



history = DIN(feature_columns, behavior_feature_list, behavior_seq_feature_list)
    
history.compile('adam',loss='binary_crossentropy', metrics=['AUC'])

history.fit(X_train, y_train, batch_size=64, epochs=1, validation_split=0.2, )

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `